In [4]:
import os
import time
# import wrds # pip install -Iv wrds==3.1.6
import pandas as pd
import numpy as np
from tqdm import tqdm
# base


## Query 작성 방법
- FROM : Library.File -> compd.secm
- Data Range : 
- https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/security-monthly/
- https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/fundamentals-quarterly/


## Code 정리

In [9]:
os.path.abspath('')

'/home/ubuntu2010/바탕화면/DEV/trading14/src/data'

In [12]:
# 실험을 위해 필요한 변수들
# nasq_ticker list : https://www.slickcharts.com/nasdaq100
# sys_dir = os.path.abspath(__file__)
BASE_DIR = os.path.abspath('').split('/src')[0]
DATA_DIR = os.path.abspath('').split('/trading')[0]+'/famafrench_data'

save_data_dir = f'{DATA_DIR}/stockdata/final_data'
nasq_ticker_name = ['AAPL', 'ADBE', 'ADI', 'ADP', 'AMAT', 'AMD', 'AMGN', 'AMZN', 'BKNG', 'COST', 
                    'CSCO', 'CSX', 'FISV', 'GILD', 'HON', 'INTC', 'INTU', 'ISRG', 'LRCX', 'MSFT', 
                    'MU', 'NFLX', 'ORLY', 'PEP', 'QCOM', 'REGN', 'SBUX', 'SNPS', 'TXN', 'VRTX']

djia_ticker_name = ['UNH', 'GS', 'MSFT', 'HD', 'MCD', 'AMGN', 'CAT', 'BA', 'HON', 
                    'TRV', 'AAPL', 'CVX', 'JNJ', 'AXP', 'PG', 'WMT', 'JPM', 'NKE', 
                    'IBM', 'MRK', 'MMM', 'DIS', 'KO', 'CSCO', 'VZ', 'INTC'] #DOW, V, WBA, CRM


os.makedirs(save_data_dir+f"/NASDAQ", exist_ok=True)
os.makedirs(save_data_dir+f"/DJIA", exist_ok=True)

In [4]:
# JSON File도 만들어야 하는데. 그거는 좀이따
portfolio_info_dir = f'{BASE_DIR}/portfolio_info'
json_name = 'portfolio.json' #'diy_portfolio.json'

os.makedirs(portfolio_info_dir+f"/NASDAQ", exist_ok=True)
os.makedirs(portfolio_info_dir+f"/DJIA", exist_ok=True)


In [5]:
# db on
db = wrds.Connection(wrds_username='kwonhj1015', wrds_password='deidala0505')

Loading library list...
Done


### Check the Database

In [6]:
db.list_libraries()
db.list_tables(library='wrdsapps')
db.describe_table(library='wrdsapps', table='firm_ratio_ibes')

Approximately 2752175 rows in wrdsapps.firm_ratio_ibes.


KeyboardInterrupt: 

In [ ]:
db.list_libraries()
db.list_tables(library='comp')
db.describe_table(library='comp', table='secd')
# stocknames = db.get_table(library='wrdsapps', table='firm_ratio', obs=20)
# stocknames2 = db.get_table(library='wrdsapps', table='firm_ratio_ibes', obs=20)
# stocknames.head()

### Code Starts

In [ ]:
def _monthly_to_daily(df:pd.DataFrame):
    #convert to datetimeindex
    df.index = pd.to_datetime(df.index, dayfirst=True)
    
    idx = df.index[-1] + pd.offsets.MonthBegin(1)
    df = df.append(df.iloc[[-1]].rename({df.index[-1]: idx}))

    #resample with forward filling values, remove last helper row
    df1 = df.resample('D').last().ffill().iloc[:-1]
    return df1

In [ ]:
# make functions

def load_crsp_data(ticker_list:list, save_dir:str, market:str='NASDAQ'):
    # set default settings
    assert market in ['NASDAQ', 'DJIA'], 'Wrong market name. Not NASDAQ OR DJIA'

    temp_dict = {}
    num_stocks =  len(ticker_list)
    ohlcv_col_name = ['date', 'tic', 'open', 'high', 'low', 'close', 'outstand_share', 'volume']
    factor_col_name = ['date', 'tic', 'BTM'] #, 'BTM'
    
    for tic in tqdm(ticker_list):
        # print(tic)
        query_ohlcv = """ SELECT datadate, tic, prcod, prchd, prcld, prccd, cshoc, cshtrd
                    FROM comp.secd
                    WHERE tic = '{}' AND
                    datadate BETWEEN '2000-01-01' AND '2023-05-01'
                    """.format(tic)
        
        query_factor = """
                    SELECT public_date, ticker, bm
                    FROM wrdsapps.firm_ratio_ibes
                    WHERE ticker = '{}' AND
                    public_date BETWEEN '2000-01-01' AND '2023-05-01'
                    """.format(tic)
        
        ohlcv_df = db.raw_sql(query_ohlcv, date_cols=['datadate'])
        factor_df = db.raw_sql(query_factor, date_cols=['datadate'])
        # preprocessing df
        ohlcv_df.columns = ohlcv_col_name
        factor_df.columns = factor_col_name
        ohlcv_df.set_index('date', inplace=True)
        factor_df.set_index('date', inplace=True)

        # factor_df's bm changes monthly, so...
        factor_df = _monthly_to_daily(factor_df)
        # ohlcv's open prices are nan
        ohlcv_df['open'] = ohlcv_df['close'].shift(1).fillna(method='bfill')

        # concat two dataframe
        new_df = pd.merge(ohlcv_df, factor_df, 
                          left_index=True, right_index=True, 
                          how='inner')
        new_df['MarketCap'] = new_df.close * new_df.outstand_share
        new_df = new_df[['open', 'high', 'low', 'close', 'volume','BTM', 'MarketCap']]
        new_df = new_df.sort_index(ascending=True)
        new_df = new_df.fillna(method='ffill')
        new_df.drop_duplicates(inplace=True)
        new_df.to_csv(f"{save_dir}/{market}/{tic}.csv", date_format='%Y-%m-%d') 
        # temp_dict[tic] = new_df
        time.sleep(1)   



In [ ]:
load_crsp_data(nasq_ticker_name, save_data_dir, market='NASDAQ')
time.sleep(1)   
load_crsp_data(djia_ticker_name, save_data_dir, market='DJIA')
# df = load_crsp_data(temp_name, save_data_dir, market='NASDAQ')


100%|██████████| 30/30 [01:58<00:00,  3.95s/it]


In [ ]:
load_crsp_data(nasq_ticker_name, save_data_dir, market='NASDAQ')
time.sleep(1)   
load_crsp_data(djia_ticker_name, save_data_dir, market='DJIA')

In [14]:
## /home/ubuntu2010/바탕화면/DEV/EIIE/Othermodel/Deep-Portfolio-Management-Reinforcement-Learning-master/mydata2/
# 서로 길이 다른 데이터가 있어 길이 통일
def sync_all_data_length(stocks:list, market:str, startdate:str='2002-09-03'):
    temp_df = pd.read_csv(save_data_dir+f'/{market}/{stocks[0]}.csv', parse_dates=['date'], index_col='date')
    temp_df = temp_df['close'].copy()
    min_len = np.inf
    start_point = None

    for s in tqdm(stocks):
        print(save_data_dir+f'/{market}/{s}.csv')
        df = pd.read_csv(save_data_dir+f'/{market}/{s}.csv', parse_dates=['date'], index_col='date')
        if min_len > len(df):
            min_len = len(df)
            start_point = df.index[0] #2002-09-03
            # print('s : ', s, ' len: ', len(df), 'Start : ', df.index[0])

        # temp_df 기준으로 data 붙여서, 인덱스 정보 통일
        tmp = pd.merge(temp_df, df, left_index=True, right_index=True, how='left', suffixes=('_tmp', ''))
        tmp.drop(['close_tmp'], axis=1, inplace=True)
        tmp = tmp.fillna(method='ffill')
        tmp = tmp.dropna(axis=0, how='all')
        tmp = tmp.loc[startdate:]
        # print(tmp.head(3))
        print(tmp.isna().sum().sum())
        print("tmp length: ", len(tmp))
        tmp.to_csv(save_data_dir+f'/{market}/{s}.csv')
    return tmp    


In [13]:
len(djia_ticker_name)

26

In [15]:
# sync_all_data_length(nasq_ticker_name, market='NASDAQ', startdate='2002-09-03')
# time.sleep(1)   
sync_all_data_length(djia_ticker_name, market='DJIA', startdate='2004-09-30')

 15%|█▌        | 4/26 [00:00<00:00, 39.58it/s]

/home1/hwang/famafrench_data/stockdata/final_data/DJIA/UNH.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/GS.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/MSFT.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/HD.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/MCD.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/AMGN.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/CAT.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/BA.csv
0
tmp length:  4596


 46%|████▌     | 12/26 [00:00<00:00, 39.16it/s]

/home1/hwang/famafrench_data/stockdata/final_data/DJIA/HON.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/TRV.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/AAPL.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/CVX.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/JNJ.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/AXP.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/PG.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/WMT.csv
0
tmp length:  4596


 77%|███████▋  | 20/26 [00:00<00:00, 39.03it/s]

/home1/hwang/famafrench_data/stockdata/final_data/DJIA/JPM.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/NKE.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/IBM.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/MRK.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/MMM.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/DIS.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/KO.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/CSCO.csv
0
tmp length:  4596


100%|██████████| 26/26 [00:00<00:00, 39.09it/s]

/home1/hwang/famafrench_data/stockdata/final_data/DJIA/VZ.csv
0
tmp length:  4596
/home1/hwang/famafrench_data/stockdata/final_data/DJIA/INTC.csv
0
tmp length:  4596


,open,high,low,close,volume,BTM,MarketCap
date,,,,,,,
2004-09-30,20.07,20.380,19.8300,20.06,72532400.0,0.227253,1.286849e+11
2004-10-01,20.06,20.920,20.2300,20.85,73831800.0,0.227253,1.337528e+11
2004-10-04,20.85,21.350,20.8500,21.13,74354650.0,0.227253,1.355490e+11
2004-10-05,21.13,21.480,20.9800,21.32,72056620.0,0.227253,1.367678e+11
2004-10-06,21.32,21.300,20.9399,21.13,57722980.0,0.227253,1.355490e+11
...,...,...,...,...,...,...,...
2022-12-23,25.97,26.190,25.6835,26.09,23587290.0,0.942580,1.076734e+11
2022-12-27,26.09,26.100,25.6500,25.94,29248590.0,0.942580,1.070544e+11
2022-12-28,25.94,26.115,25.5000,25.54,26079790.0,0.942580,1.054036e+11


In [ ]:
# 모든 코드 끝났을때
db.close()

# Draw & Save Time series Graphs 

In [1]:
import os
import time
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
BASE_DIR = os.path.abspath('').split('/src')[0]
DATA_DIR = os.path.abspath('').split('/trading')[0]+'/famafrench_data'

save_data_dir = f'{DATA_DIR}/stockdata/final_data'
nasq_ticker_name = ['MSFT', 'AAPL', 'AMZN', 'META', 'GOOG', 'TSLA', 'PEP','AVGO', 'COST', 'CSCO', 
                    'TMUS', 'CMCSA', 'ADBE', 'TXN', 'NFLX', 'AMD', 'HON', 'SBUX', 'QCOM', 'INTC', 
                    'AMGN', 'INTU', "ISRG", 'MDLZ', 'GILD', 'BKNG', 'AMAT', 'ADP', 'ADI', 'VRTX'] # nasdaq top 30 

djia_ticker_name = ['UNH', 'GS', 'MSFT', 'HD', 'MCD', 'AMGN', 'V', 'CAT', 'BA', 'HON', 
                    'CRM', 'TRV', 'AAPL', 'CVX', 'JNJ', 'AXP', 'PG', 'WMT', 'JPM', 'NKE', 
                    'IBM', 'MRK', 'MMM', 'DIS', 'KO', 'DOW', 'CSCO', 'VZ', 'WBA', 'INTC']


market = 'NASDAQ' # 'DJIA'
tic = 'AAPL'
file_dir = f"{save_data_dir}/{market}/" #f"{save_data_dir}/{market}/{tic}.csv"

In [ ]:
def draw_plots(ticker_name:list, market):
    
    
    
    
    return